In [1]:
# -*- coding: utf-8 -*-
"""Training_Sony.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1GEBhrMgdTE81EYLi2hUD6Xrie9iRhLZg
"""
from __future__ import division
import os, time

#pip install scipy==1.2.0

import scipy as scipy

# uniform content loss + adaptive threshold + per_class_input + recursive G
# improvement upon cqf37

#from google.colab import drive
#drive.mount('/content/drive',force_remount = True)

import tensorflow.compat.v1 as tf
#import tensorflow.contrib.slim as slim
import numpy as np

#!pip install --upgrade tf_slim

import tf_slim as slim

import sys

#pip install rawpy

import rawpy as rawpy
import glob
from PIL import Image

2022-11-23 01:43:14.045322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 01:43:14.234165: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-23 01:43:14.234197: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-23 01:43:14.272718: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-23 01:43:14.936609: W tensorflow/stream_executor/platform/de

In [2]:

input_dir = '/home/21i190003/DL_Project/Sony_Dataset/Sony/short/'
gt_dir = '/home/21i190003/DL_Project/Sony_Dataset/Sony/long/'
checkpoint_dir = '/home/21i190003/DL_Project/Sony_SSIM/Sony_ssim_ckpt/'
result_dir = '/home/21i190003/DL_Project/Sony_SSIM/Sony_ssim_result/'
print('------------------------------------------------------------------\n------------------------------------------------------------------\n------------------------------------------------------------------')        

# get train IDs
train_fns = glob.glob(gt_dir + '0*.ARW')
train_ids = [int(os.path.basename(train_fn)[0:5]) for train_fn in train_fns]

print(f'No. of Training Images: {len(train_ids)}') #Number of reference images for training for Sony Sensor

ps = 512  # patch size for training
save_freq = 5

DEBUG = 0
if DEBUG == 1:
    save_freq = 1
    train_ids = train_ids[0:5]


------------------------------------------------------------------
------------------------------------------------------------------
------------------------------------------------------------------
No. of Training Images: 161


In [4]:
def lrelu(x):
    return tf.maximum(x * 0.2, x)

def upsample_and_concat(x1, x2, output_channels, in_channels):
    pool_size = 2
    deconv_filter = tf.Variable(tf.truncated_normal([pool_size, pool_size, output_channels, in_channels], stddev=0.02))
    deconv = tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2), strides=[1, pool_size, pool_size, 1])

    deconv_output = tf.concat([deconv, x2], 3)
    deconv_output.set_shape([None, None, None, output_channels * 2])

    return deconv_output


def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out


In [5]:
def network(input):
    conv1 = slim.conv2d(input, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_1')
    #print(conv1.shape)
    conv1 = slim.conv2d(conv1, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_2')
    #print(conv1.shape)
    pool1 = slim.max_pool2d(conv1, [2, 2], padding='SAME')
    #print(pool1.shape)
    conv2 = slim.conv2d(pool1, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_1')
    #print(conv2.shape)
    conv2 = slim.conv2d(conv2, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_2')
    #print(conv2.shape)
    pool2 = slim.max_pool2d(conv2, [2, 2], padding='SAME')
    #print(pool2.shape)    
    conv3 = slim.conv2d(pool2, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_1')
    conv3 = slim.conv2d(conv3, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_2')
    pool3 = slim.max_pool2d(conv3, [2, 2], padding='SAME')

    conv4 = slim.conv2d(pool3, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_1')
    conv4 = slim.conv2d(conv4, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_2')
    pool4 = slim.max_pool2d(conv4, [2, 2], padding='SAME')

    conv5 = slim.conv2d(pool4, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_1')
    conv5 = slim.conv2d(conv5, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_2')
    #print(conv5.shape)
    up6 = upsample_and_concat(conv5, conv4, 256, 512)
    conv6 = slim.conv2d(up6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_1')
    conv6 = slim.conv2d(conv6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_2')

    up7 = upsample_and_concat(conv6, conv3, 128, 256)
    conv7 = slim.conv2d(up7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_1')
    conv7 = slim.conv2d(conv7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_2')

    up8 = upsample_and_concat(conv7, conv2, 64, 128)
    conv8 = slim.conv2d(up8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_1')
    conv8 = slim.conv2d(conv8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_2')

    up9 = upsample_and_concat(conv8, conv1, 32, 64)
    conv9 = slim.conv2d(up9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_1')
    conv9 = slim.conv2d(conv9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_2')

    conv10 = slim.conv2d(conv9, 12, [1, 1], rate=1, activation_fn=None, scope='g_conv10')
    #print(conv10.shape)
    conv10 = slim.conv2d(conv10, 3, [2, 2], rate=1, activation_fn=None, scope='g_conv10_1')
    out = conv10
    print(input.shape)
    print(out.shape)
    #out = tf.depth_to_space(conv10, 2)
    #print('Shape of U-Net output: ',out.shape)
    return out

array([0])

In [6]:
print('Initiating Session....')
init_op = tf.initialize_all_variables()
sess = tf.Session()
tf.disable_eager_execution()
in_image = tf.placeholder(tf.float32, [None, None, None, 4])
gt_image = tf.placeholder(tf.float32, [None, None, None, 3])
tf.disable_v2_behavior()
out_image = network(in_image)
#print(out_image.shape)
#G_loss = tf.reduce_mean(tf.abs(out_image - gt_image))
G_loss = tf.reduce_mean(tf.image.ssim(out_image, gt_image, 1.0))
t_vars = tf.trainable_variables()
lr = tf.placeholder(tf.float32)
G_opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss)

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
if ckpt:
    print('loaded ' + ckpt.model_checkpoint_path)
    saver.restore(sess, ckpt.model_checkpoint_path)


# Raw data takes long time to load. Keep them in memory after loaded.
gt_images = [None] * 6000
input_images = {}
input_images['300'] = [None] * len(train_ids)
input_images['250'] = [None] * len(train_ids)
input_images['100'] = [None] * len(train_ids)

g_loss = np.zeros((5000, 1))

allfolders = glob.glob(result_dir + '*0')


Initiating Session....
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
non-resource variables are not supported in the long term


2022-11-23 01:44:22.097328: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-23 01:44:22.097392: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-23 01:44:22.097425: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kopri): /proc/driver/nvidia/version does not exist
2022-11-23 01:44:22.097880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/21i190003/.local/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning:

(?, ?, ?, 4)
(?, ?, ?, 3)


2022-11-23 01:44:23.815452: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [5]:
save_freq=1

In [8]:
print('Training Started------------------------------------------------------------------\n------------------------------------------------------------------\n------------------------------------------------------------------')        
np.random.seed(10)
lastepoch = 0
for folder in allfolders:
    lastepoch = np.maximum(lastepoch, int(folder[-4:]))

import warnings
warnings.filterwarnings("ignore")

learning_rate = 1e-4
for epoch in range(lastepoch, 3):
    if os.path.isdir(result_dir + '%04d' % epoch):
        continue
    cnt = 0
    if epoch > 2000:
        learning_rate = 1e-5

    for ind in np.random.permutation(len(train_ids)):
        # get the path from image id
        train_id = train_ids[ind]
        in_files = glob.glob(input_dir + '%05d_00*.ARW' % train_id)
        in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]
        in_fn = os.path.basename(in_path)

        gt_files = glob.glob(gt_dir + '%05d_00*.ARW' % train_id)
        gt_path = gt_files[0]
        gt_fn = os.path.basename(gt_path)
        in_exposure = float(in_fn[9:-5])
        gt_exposure = float(gt_fn[9:-5])
        ratio = min(gt_exposure / in_exposure, 300)

        st = time.time()
        cnt += 1

        if input_images[str(ratio)[0:3]][ind] is None:
            #try:
                raw = rawpy.imread(in_path)
                input_images[str(ratio)[0:3]][ind] = np.expand_dims(pack_raw(raw), axis=0) * ratio

                gt_raw = rawpy.imread(gt_path)
                im = gt_raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
                gt_images[ind] = np.expand_dims(np.float32(im / 65535.0), axis=0)
        # crop
        H = input_images[str(ratio)[0:3]][ind].shape[1]
        W = input_images[str(ratio)[0:3]][ind].shape[2]

        xx = np.random.randint(0, W - ps)
        yy = np.random.randint(0, H - ps)
        input_patch = input_images[str(ratio)[0:3]][ind][:, yy:yy + ps, xx:xx + ps, :]
        gt_patch = gt_images[ind][:, yy * 2:yy * 2 + ps * 2, xx * 2:xx * 2 + ps * 2, :]

        if np.random.randint(2, size=1)[0] == 1:  # random flip
            input_patch = np.flip(input_patch, axis=1)
            gt_patch = np.flip(gt_patch, axis=1)
        if np.random.randint(2, size=1)[0] == 1:
                    input_patch = np.flip(input_patch, axis=2)
                    gt_patch = np.flip(gt_patch, axis=2)
        if np.random.randint(2, size=1)[0] == 1:  # random transpose
                    input_patch = np.transpose(input_patch, (0, 2, 1, 3))
                    gt_patch = np.transpose(gt_patch, (0, 2, 1, 3))

        input_patch = np.minimum(input_patch, 1.0)

        _, G_current, output = sess.run([G_opt, G_loss, out_image],
                                                feed_dict={in_image: input_patch, gt_image: gt_patch, lr: learning_rate})
        output = np.minimum(np.maximum(output, 0), 1)
        g_loss[ind] = G_current
        #print(g_loss[np.where(g_loss)])
        print("%d , Count:  %d , Train ID: %d , Loss=%.3f , Time=%.3f" % ((epoch+1), cnt,train_id, np.mean(g_loss[np.where(g_loss)]), time.time() - st))

        if epoch % save_freq == 0:
            if not os.path.isdir(result_dir + '%04d' % epoch):
                os.makedirs(result_dir + '%04d' % epoch)

            temp = np.concatenate((gt_patch[0, :, :, :], output[0, :, :, :]), axis=1)
            Image.fromarray((temp * 255).astype(np.uint8),mode='RGB').save(
                        result_dir + '%04d/%05d_00_train_%d.jpg' % (epoch, train_id, ratio))        
        #except:
        #    print(f"Oops! {sys.exc_info()[0]} occurred for Train ID {train_id}, moving to next training ID....")

        #if cnt>=100:
        #    break
    print(f'------------------------------------------------------------------\nEpoch: {epoch+1} to Epoch: {epoch+2}------------------------------------------------------------------\n------------------------------------------------------------------')        
    if not os.path.isdir(result_dir + 'model.ckpt'):
      os.makedirs(result_dir + 'model.ckpt')
    saver.save(sess, checkpoint_dir + 'model.ckpt')


Training Started------------------------------------------------------------------
------------------------------------------------------------------
------------------------------------------------------------------
2 , Count:  1 , Train ID: 42 , Loss=-0.042 , Time=3.787
2 , Count:  2 , Train ID: 24 , Loss=-0.060 , Time=3.695
2 , Count:  3 , Train ID: 150 , Loss=-0.091 , Time=4.778
2 , Count:  4 , Train ID: 44 , Loss=-0.085 , Time=4.709
2 , Count:  5 , Train ID: 196 , Loss=-0.087 , Time=4.799
2 , Count:  6 , Train ID: 97 , Loss=-0.096 , Time=4.742
2 , Count:  7 , Train ID: 118 , Loss=-0.090 , Time=4.696
2 , Count:  8 , Train ID: 85 , Loss=-0.087 , Time=4.736
2 , Count:  9 , Train ID: 17 , Loss=-0.093 , Time=4.699
2 , Count:  10 , Train ID: 181 , Loss=-0.092 , Time=4.713
2 , Count:  11 , Train ID: 38 , Loss=-0.095 , Time=4.692
2 , Count:  12 , Train ID: 138 , Loss=-0.096 , Time=4.730
2 , Count:  13 , Train ID: 15 , Loss=-0.105 , Time=4.591
2 , Count:  14 , Train ID: 123 , Loss=-0.118 ,

KeyboardInterrupt: 